In [1]:
import pandas as pd

In [2]:
vehicles_df = pd.read_csv('data/vehicles.csv')
demand_df = pd.read_csv('data/demand.csv')

In [3]:
vehicles_df

,ID,Vehicle,Size,Year,Cost ($),Yearly range (km),Distance
0,BEV_S1_2023,BEV,S1,2023,187000,102000,D1
1,BEV_S1_2024,BEV,S1,2024,177650,102000,D1
2,BEV_S1_2025,BEV,S1,2025,168767,102000,D1
3,BEV_S1_2026,BEV,S1,2026,160329,102000,D2
4,BEV_S1_2027,BEV,S1,2027,152312,102000,D2
...,...,...,...,...,...,...,...
187,LNG_S3_2034,LNG,S3,2034,209208,73000,D4
188,LNG_S3_2035,LNG,S3,2035,215484,73000,D4
189,LNG_S3_2036,LNG,S3,2036,221948,73000,D4
190,LNG_S3_2037,LNG,S3,2037,228607,73000,D4


In [4]:
demand_df

,Year,Size,Distance,Demand (km)
0,2023,S1,D1,869181
1,2023,S1,D2,2597094
2,2023,S1,D3,3292011
3,2023,S1,D4,414315
4,2023,S2,D1,995694
...,...,...,...,...
251,2038,S4,D4,2446
252,2038,S3,D1,3229278
253,2038,S3,D2,3297618
254,2038,S3,D3,1448550


In [20]:
def allocate_vehicles(demand_df, vehicles_df, year):
    # Mapping for categorical distance values
    distance_mapping = {"D1": 1, "D2": 2, "D3": 3, "D4": 4}
    
    # Convert categorical distance to numeric ranks
    vehicles_df['Distance_categorical'] = vehicles_df['Distance'].map(distance_mapping)
    demand_df['Distance_categorical'] = demand_df['Distance'].map(distance_mapping)

    demand_df = demand_df[demand_df['Year'] == year].copy() # Filter relevant year
    available_vehicles = vehicles_df[vehicles_df['Year'] <= year]  # Vehicles available in that year

    def find_allocations(row):
        size_needed = row['Size']
        distance_needed = row['Distance_categorical']
        
        allocated_vehicles = available_vehicles[
            (available_vehicles['Size'] == size_needed) & 
            (available_vehicles['Distance_categorical'] <= distance_needed)
        ]['ID'].tolist() 
        
        return allocated_vehicles

    demand_df['Allocation'] = demand_df.apply(find_allocations, axis=1)  # Apply function row-wise
    
    return demand_df


In [21]:
allocation_df = allocate_vehicles(demand_df, vehicles_df, 2023)
allocation_df.to_csv('data/allocation_result.csv', index=False)